# Poem Generation Using Web-Scraped Text

## The goal of this project is to generate (decent) poems from scraped website text using a keyowrd as an input.

In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np

In [2]:
tokenizer = Tokenizer()

text = open('poem.txt',encoding="utf8").read()

# Split threats each line as a sentence.
corpus = text.lower().split("\n")

tokenizer.fit_on_texts(corpus)

total_words = len(tokenizer.word_index) + 1

In [3]:
# Training Data

input_sequences = []

for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)
        
# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])

input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding='pre'))


# Create predictors and label
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]

label = ku.to_categorical(label, num_classes = total_words)

In [4]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 152, 100)          416100    
_________________________________________________________________
bidirectional (Bidirectional (None, 152, 300)          301200    
_________________________________________________________________
dropout (Dropout)            (None, 152, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 2080)              210080    
_________________________________________________________________
dense_1 (Dense)              (None, 4161)              8659041   
Total params: 9,746,821
Trainable params: 9,746,821
Non-trainable params: 0
______________________________________________

In [5]:
#taking long time given epoch count, need to look into this and adjusting for repetitive word use in outputs when trained with only 10 epochs

history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
571/571 [==============================] - 19s 33ms/step - loss: 7.0924 - accuracy: 0.0698
Epoch 2/100
571/571 [==============================] - 19s 34ms/step - loss: 6.5236 - accuracy: 0.0713
Epoch 3/100
571/571 [==============================] - 19s 33ms/step - loss: 6.3136 - accuracy: 0.0867
Epoch 4/100
571/571 [==============================] - 19s 34ms/step - loss: 6.1629 - accuracy: 0.0936
Epoch 5/100
571/571 [==============================] - 19s 34ms/step - loss: 5.9840 - accuracy: 0.1005
Epoch 6/100
571/571 [==============================] - 19s 33ms/step - loss: 5.8031 - accuracy: 0.1085
Epoch 7/100
571/571 [==============================] - 19s 34ms/step - loss: 5.6510 - accuracy: 0.1161
Epoch 8/100
571/571 [==============================] - 19s 33ms/step - loss: 5.5278 - accuracy: 0.1209
Epoch 9/100
571/571 [==============================] - 19s 33ms/step - loss: 5.4199 - accuracy: 0.1258
Epoch 10/100
571/571 [==============================] - 19s 33ms/step - l

In [14]:
seed_text = "world"
next_words = 50
  
for _ in range(next_words):
 token_list = tokenizer.texts_to_sequences([seed_text])[0]
 token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
 predicted = model.predict_classes(token_list, verbose=0)
 output_word = ""
 for word, index in tokenizer.word_index.items():
  if index == predicted:
   output_word = word
   break
 seed_text += " " + output_word
print(seed_text)

world it deaf and macbeth john purgatory satirical assay of the heather attached hollows of sonnets said he came and tulips by sylvia plath two remarkable poems by the golden haired rock star of american poetry they had left the lady of shallot by alfred lord tennyson rime of the night
